### Imports

In [10]:
import onnx
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image
import onnxruntime as ort

## Question 1

In [13]:
model = onnx.load("hair_classifier_v1.onnx")
graph = model.graph

input_names = [inp.name for inp in graph.input]
output_names = [out.name for out in graph.output]

print("Inputs:", input_names)
print("Outputs:", output_names)

Inputs: ['input']
Outputs: ['output']


## Question 2

In [14]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

mean= np.array([0.485, 0.456, 0.406]).reshape(3, 1, 1)
std  = np.array([0.229, 0.224, 0.225]).reshape(3, 1, 1)

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    img = np.array(img).astype("float32") / 255.0 
    img = np.transpose(img, (2, 0, 1))

    img = (img - mean) / std

    return img

In [15]:
img = download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")
img = prepare_image(img,(200,200))

img[0,0,0]

np.float64(-1.0732939435925546)

## Question 3

In [16]:
session = ort.InferenceSession("hair_classifier_v1.onnx")
img_batch = np.expand_dims(img, axis=0).astype(np.float32)
outputs = session.run(None, {input_names[0]: img_batch})
outputs[0]

array([[0.09156627]], dtype=float32)